In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.2-spark3.2-s_2.12 pyspark-shell'

In [ ]:
import sys

from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame, Column

from typing import Callable, Tuple

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Pregel")
        .master("local[4]")
        .getOrCreate()
)
sc = spark.sparkContext

# Pregel

In [ ]:
def enrich_deg(vertices: DataFrame, edges: DataFrame):
    out_deg = (
        vertices
            .join(edges, col("id") == col("src"))
            .groupby("id").count()
            .withColumn("out_deg", col("count"))
            .drop("count")
        ).cache()

    in_deg = (
        vertices
            .join(edges, col("id") == col("dst"))
            .groupby("id").count()
            .withColumn("in_deg", col("count"))
            .drop("count")
        ).cache()

    return (
        vertices
            .join(out_deg, "id", "left")
                .withColumn("out_deg", F.coalesce("out_deg", F.lit(0)))
            .join(in_deg, "id", "left")
                .withColumn("in_deg", F.coalesce("in_deg", F.lit(0)))
            .orderBy("id")
        ).cache()

## Компоненты связности графа

![graph](./imgs/graphs-connected-components.drawio.svg)

In [ ]:
def connected_components_graph() -> Tuple[DataFrame, DataFrame]:
    verties = [ (x, x) for x in range(1, 7) ]
    verties_df = (
        spark
            .createDataFrame(verties)
            .toDF("id", "value")
    )
    edges = [ (1, 2), (1,3), (2, 3), (3, 4), (4,5), (6, 6) ]
    edges_df = (
        spark
            .createDataFrame(edges)
            .toDF("src", "dst")
    )
    return enrich_deg(verties_df, edges_df), edges_df

In [ ]:
verties_df, edges_df = connected_components_graph()

In [ ]:
for _ in range(5):
    verties_df = (
        edges_df.join(verties_df, col("src") == col("id"))
          .select(col("dst").alias("id"), col("value"))
          .groupby(col("id")).agg(F.min("value").alias("message"))
          .join(verties_df, "id", "right")
          .select("id", F.coalesce("message", "value").alias("message"), "value")
          .select("id", F.least("message", "value").alias("value"))
    )
verties_df.show()

## Минимальное расстояние

![graph](./imgs/graphs-min-dist.drawio.svg)

In [ ]:
def min_dist_graph(start: int) -> Tuple[DataFrame, DataFrame]:
    verties = [ (x, 0 if x == start else sys.maxsize // 2) for x in range(1, 6) ]
    verties_df = (
        spark
            .createDataFrame(verties)
            .toDF("id", "value")
    )
    edges = [ (1, 2, 1), (1, 3, 3), (2, 3, 1), (3, 4, 3), (4, 5, 4) ]
    edges_df = (
        spark
            .createDataFrame(edges)
            .toDF("src", "dst", "weight")
    )
    return enrich_deg(verties_df, edges_df), edges_df

In [ ]:
verties_df, edges_df = min_dist_graph(start = 1)

In [ ]:
for _ in range(5):
    verties_df = (
        edges_df
            .join(verties_df, col("src") == col("id"))
            .select(col("dst").alias("id"), F.expr("value + weight").alias("value"))
            .groupby("id").agg(F.min("value").alias("message"))
            .join(verties_df, "id", "right")
            .select("id", F.coalesce("message", "value").alias("message"), "value")
            .select("id", F.least("message", "value").alias("value"))
    )
verties_df.show()

## PageRank

![graph](./imgs/graphs-page-rank.png)

In [ ]:
def pagerank_graph() -> Tuple[DataFrame, DataFrame]:
    vertices = [ (chr(ord('A') + x), 1) for x in range(11) ]
    verties_df = (
        spark
            .createDataFrame(vertices)
            .toDF("id", "value")
    )

    edges = [
        ('D', 'A'),
        ('D', 'B'), ('E', 'B'), ('F', 'B'), ('C', 'B'), ('G', 'B'), ('H', 'B'), ('I', 'B'),
        ('E', 'D'), ('E', 'F'), ('F', 'E'),
        ('G', 'E'), ('H', 'E'), ('I', 'E'), ('J', 'E'), ('K', 'E'),
        ('B', 'C')
    ]
    edges_df = (
        spark
            .createDataFrame(edges)
            .toDF("src", "dst")
    )
    
    return enrich_deg(verties_df, edges_df), edges_df

In [ ]:
verties_df, edges_df = pagerank_graph()

In [ ]:
verties_df1 = verties_df

In [ ]:
out_deg_col = F.when(col("out_deg") == 0, 1).otherwise(col("out_deg")).alias("out_deg")
verties_df1 = (
        edges_df
            .join(verties_df, col("src") == col("id"))
            .select(col("dst").alias("id"), col("value"))
            .groupby("id").agg(F.sum(col("value")).alias("message"))
            .join(verties_df, "id", "right")
            .select("id", out_deg_col, F.coalesce("message", "value").alias("message"), "value")
            .select("id", F.expr("(0.15 + 0.85 * message) / out_deg").alias("value"))
    )

verties_df1.show()

In [ ]:
spark.sparkContext.setCheckpointDir("plan/checkpoint")

for _ in range(10):
    verties_df = (
        edges_df
            .join(verties_df, col("src") == col("id"))
            .select(col("dst").alias("id"), col("value"))
            .groupby("id").agg(F.sum(col("value")).alias("message"))
            .join(verties_df, "id", "right")
            .select("id", out_deg_col, F.coalesce("message", "value").alias("message"), "value")
            .select("id", out_deg_col, F.expr("(0.15 + 0.85 * message) / out_deg").alias("value"))
    ).checkpoint()

In [ ]:
verties_df.orderBy("id").show()

## Алгоритм параллельной обработки графов Прегель (Pregel)

In [ ]:
def pregel_superstep(vertices: DataFrame, edges: DataFrame, message: Column, message_merger: Callable[Column, Column], value: Column):
    out_deg_col = (
        F.when(
            col("out_deg") == 0,
            1
        )
        .otherwise(col("out_deg"))
    ).alias("out_deg")

    return (
        edges
            .join(vertices, col("src") == col("id"))
            .select(col("dst").alias("id"), message.alias("value"))
            .groupby("id").agg(message_merger(col("value")).alias("message"))
            .join(vertices, "id", "right")
            .select("id", out_deg_col, F.coalesce("message", "value").alias("message"), "value")
            .select("id", out_deg_col, value.alias("value"))
    )

### Компоненты связности

In [ ]:
verties_df, edges_df = connected_components_graph()

In [ ]:
for _ in range(5):
    verties_df = pregel_superstep(
        verties_df,
        edges_df,
        col("value"),
        F.min,
        F.least("message", "value")
    )

In [ ]:
verties_df.show()

### Минимальное расстояние

In [ ]:
verties_df, edges_df = min_dist_graph(start = 1)

In [ ]:
for _ in range(5):
    verties_df = pregel_superstep(
        verties_df,
        edges_df,
        F.expr("value + weight"),
        F.min,
        F.least("message", "value")
    )

In [ ]:
verties_df.show()

### PageRank

In [ ]:
verties_df, edges_df = pagerank_graph()

In [ ]:
for _ in range(10):
    verties_df = pregel_superstep(
        verties_df,
        edges_df,
        col("value"),
        F.sum,
        F.expr("(0.15 + 0.85 * message) / out_deg")
    ).checkpoint()

In [ ]:
verties_df.orderBy("id").show()

## Домашнее задание

1. Реализовать сумму вершин графа при помощи `pregel_superstep`. По окончанию алгоритма все вершины графа должны иметь одно значение, которое будет являться суммой всех вершин графа. Игнорировать графы, в которых существуют несвязные подграфы
1. Реализовать алгоритм поиска максимального значения при помощи `pregel_superstep`. По окончанию алгоритма все вершины графа должны иметь одно значение, которое будет являться максимальным значением среди всех значений вершин графа. Игнорировать графы, в которых существуют несвязные подграфы

# GraphFrames

## Создание GraphFrame

Пользователи могут создавать GraphFrames из двух датафреймов: вершины (vertex) и ребра (edge).

- датафрейм с вершинами должен иметь колонку по имени `id`, которая является уникальным ID для каждой вершины в графе
- датафрейм с ребрами должен содержать две колонки `src` - id исходной вершины, `dst` - id конечной вершины

Оба датафрейма могут содержать произвольное количество дополнительных колонок.


In [ ]:
from functools import reduce

from graphframes import *
from graphframes.lib import Pregel

In [ ]:
vertices = (
    spark.createDataFrame([
      ("a", "Alice", 34),
      ("b", "Bob", 36),
      ("c", "Charlie", 30),
      ("d", "David", 29),
      ("e", "Esther", 32),
      ("f", "Fanny", 36),
      ("g", "Gabby", 60)
    ])
    .toDF("id", "name", "age")
)

In [ ]:
edges = (
    spark.createDataFrame([
      ("a", "b", "friend"),
      ("b", "c", "follow"),
      ("c", "b", "follow"),
      ("f", "c", "follow"),
      ("e", "f", "follow"),
      ("e", "d", "friend"),
      ("d", "a", "friend"),
      ("a", "e", "friend")
    ])
    .toDF("src", "dst", "relationship")
)

In [ ]:
g = GraphFrame(vertices, edges)

## Основные запросы

In [ ]:
g.inDegrees.show()

In [ ]:
g.outDegrees.show()

In [ ]:
g.degrees.show()

Можно выполнять запросы напрямую на ребрах графа:

In [ ]:
g.edges.filter("relationship = 'follow'").count()

или на вершинах

In [ ]:
g.vertices.groupBy().min("age").show()

## Язык запросов Motif

Motif позволяет строить сложные запросы по связям между вершинами и ребрами. Например, можно получить пары вершин, которые имеют двунаправленные ребра между ними:

In [ ]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")

Результатом вычисления является новый DataFrame, в котором имена колонок совпадают по именам ключей из motif запроса:

In [ ]:
type(motifs)

In [ ]:
motifs.printSchema()

In [ ]:
motifs.show()

Получив в результате применения motif-запроса датафрем, можно использовать все операции, доступные DataFrame. Например, можно получить связи, в которых один возраст человека в одной из вершин больше 30 лет:

In [ ]:
motifs.filter("b.age > 30 or a.age > 30").show()

### Запросы с сохранением состояния (stateful)

Большинство motif-запросов выполняются без сохранения состояния (stateless), которые можно легко выразить при помощи примеров выше. Следующий пример демонстрирует более сложные запросы, которые переносят состояния по путям в motif-запросах. Такие запросы могут быть выражены при помощи комбинации motif-запросов из GraphFrame и фильтров примененных к результирующему DataFrame.

Например, необходимо получить четыре вершины, с некоторым свойством, которое определяется последовательностью функций. Так, по цепочке четырех вершин `a->b->c->d` необходимо получить подмножество цепочек, которые отвечают условиям фильтра:

Инициализировать состояние
Обновить состояние на базе значения из вершины `a`
Обновить состояние на базе значения из вершины `b`
Обновить состояние на базе значения из вершины `c`, `d`, и т.д.

Если конечное состояние проходит проверку по условию, тогда такая цепочка пропускается. Ниже приведен пример кода, который демонстрирует этот процесс: выделяются цепочка из четырех вершин, у которых как минимум два ребра из трех имеют значение `"friend"`:

In [ ]:
chain4 = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")

In [ ]:
def accumulateFriends(cnt, edge):
    relationship = col(edge)["relationship"]
    return F.when(relationship == "friend", cnt + 1).otherwise(cnt)

In [ ]:
edges = ["ab", "bc", "cd"]
numFriends = reduce(accumulateFriends, edges, F.lit(0))

In [ ]:
chainWith2Friends2 = (
    chain4.withColumn("num_friends", numFriends)
        .where(numFriends >= 2)
)
chainWith2Friends2.show()

## Подграфы

GraphFrames предлагает API для построяния подграфов через фильтрацию вершин и ребрер. Эти фильтры могут быть объеденены, например, следующий подграф содержит только людей, которые:

- старше 30 лет
- имеют друзей, которые так же старше 30

In [ ]:
g2 = (
    g.filterEdges("relationship = 'friend'")
    .filterVertices("age > 30")
    .dropIsolatedVertices()
)

In [ ]:
g2.vertices.show()

In [ ]:
g2.edges.show()

## Стандартные алгоритмы на графах

GraphFrames предлагает готовые алгоритмы стандартной для обработки графов:

- Поиск в ширину (Breadth-first search (BFS))
- Связные компоненты (Connected components)
- Строго связные компоненты (Strongly connected components)
- Алгоритм распространения меток (Label Propagation Algorithm (LPA))
- PageRank
- Кратчайщий путь (Shortest paths)
- Количество треугольников (Triangle count)

In [ ]:
paths = g.bfs("name = 'Esther'", "age < 32")
paths.show()

можно применить фильтр только к началу или концу ребра:

In [ ]:
filteredPaths = g.bfs(
  fromExpr = "name = 'Esther'",
  toExpr = "age < 32",
  edgeFilter = "relationship != 'friend'",
  maxPathLength = 3)

filteredPaths.show()

In [ ]:
sc.setCheckpointDir("/tmp/graphframes-example-connected-components")

result = g.connectedComponents()

In [ ]:
result.show()

In [ ]:
result = g.stronglyConnectedComponents(maxIter=10)

result.select("id", "component").show()

Распространение меток можно использовать для того, чтобы найти "сообщества" в графе.

Каждый узел в графе изначально является своим собственным "сообществом". На каждом шаге алгоритма (superstep), узлы посылают сообщения своим соседям о том, к какому "сообществу" они принадлежат, а также обновляют свою принадлежность к сообществу на базе сообщий, которые были отправлены им их соседями.

LPA является стандартным алгоритмом для определения "сообществ" в графе. Хотя алгоритм является нетребовательным к ресурсам, но:

- Алгоритм не гарантирует, что он сойдется (см. сходимость/расходимость алгоримов)
- Алгоритм может остановиться на шаге, когда все вершины графа являются членами своего собственного "сообщества"

In [ ]:
result = g.labelPropagation(maxIter=5)
result.show()

In [ ]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.show()
results.edges.show()

In [ ]:
# PageRank выполняет фиксированное число шагов
results = g.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.orderBy("id").show()
results.edges.show()

In [ ]:
# Запустить PageRank, конкретно для вершины `a`

results = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="a")
results.vertices.show()
results.edges.show()

In [ ]:
results = g.shortestPaths(landmarks=["a", "d"])
results.show()

In [ ]:
results = g.triangleCount()
results.show()

In [ ]:
spark.catalog.clearCache()

## Разработка своих алгоритмов на базе GraphFrames

Свои алгоритмы можно разрабатывать при помощи [`GraphFrame.pregel`](https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html?highlight=pregel#graphframes.GraphFrame.pregel)

In [ ]:
alpha = 0.15
numVertices = g.vertices.count()

initialMsg = F.lit(1.0 / numVertices)
afterMsgAgg = F.coalesce(Pregel.msg(), F.lit(0.0)) * F.lit(1.0 - alpha) + F.lit(alpha / numVertices)

graph = GraphFrame(g.outDegrees, g.edges)

ranks = (
    graph.pregel
    .setMaxIter(10)
    # withVertexColumn создает дополнительную колонку, в которой будет накапливаться результат
    .withVertexColumn(
        "rank", # в этом столбце будет накапливаться PageRank
        initialMsg, # начальное значение столбца
        afterMsgAgg # механизм обновления столбца
    )
    # sendMsgToDst определяет как формируется сообщение, которое необходимо отправить каждому соседу (по направлению ребра)
    .sendMsgToDst(Pregel.src("rank") / Pregel.src("outDegree"))
    # aggMsgs определяет как сообщения будут аггрегироваться для конкретной вершины
    .aggMsgs(F.sum(Pregel.msg()))
)


In [ ]:
g.outDegrees.show()

In [ ]:
result = ranks.run()

In [ ]:
result.orderBy("id").show()